Updated with Bands informaiton for V3.

# Land Cover Classification:

This study consists of 5 mauzas (Revenue Estates) of tehsil Pabbi, district Nowshera, Khyber Pakhtunkhwa Pakistan. These five reveue states are
1. Khushmaqam
2. Khudrezi
3. Chowki Mamraiz
4. Chand Bibi
5. Amankot

The original land use mentoined in the Landuse_Ma columns of the massavi is out dated and donot match with the ground realities. The land use and total number of parcels in the given dataset are 

| Landuse_Ma     | parcel_count | total_area_acre  |
|----------------|--------------|------------------|
| Agriculture    | 5759         | 3878.35          |
| Built up       | 24           | 73.87            |
| Graveyard      | 29           | 25.20            |
| Other          | 229          | 130.85           |
| Road/Streets   | 172          | 207.78           |
| Stream         | 302          | 102.74           |
|Total           | 6515         | 4419             |


Some of the anomalies in this dataset are as follow:
- The type mentioned in Land use column as agricultue has been converted into builtup, and therefore, not suitable for crop classification dataset.
- The type mentioned in Land use column as agricultue has now both builtup and agriculture, should be cleaned to make it suiutable for crop classification.
- Some of the on ground agriculture parcels has been recorded as other in Land use column maynot participate in crop classificaiton dataset as per this dataset.

## Purpose
The purpose of this exercise is to get clean agriculture parcels dataset for crop classificaiton using artifical intelligence algorithms like random forest, SVM etc. 

## Data Cleansing and Preperation Methodology:
Since the agriculture type in the Land use column does not give us a clear idea of the current land type therefore, we have to use other means to computer the current land use type. Therefore, as first step we will filter the 5,759 agriculter parcels to be included in the study. Since our focus is on agriculture and crops detection therefore, the rest of the parcels will filterout from this study and we will clean the 5,759 agricultuer parcels. The data pipeline shows step by step processing of model.

![Fig: Model Pipeline ](./Model-V2.png)


### Imagery Acquisiton and Processing
1. Acquired Harmonized Sentinel-2 imagery [COPERNICUS/S2_SR_HARMONIZED](https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2_SR_HARMONIZED) using Earth
   Engine for the time peroid (2022-2024). In this case 419 images were acquired with cloud cover less than 10%. This imagery was processed to calculate the features (NDVI etc.) of the three years (2022-2024) 419 images using Google Earth Engine. This timeseriese mean of three years, 419 images is aggregated into one mean composite image which in simple words is a mean of means. These features will be used initially for manual cleaning to get a sample label dataset. The label dataset will be used to train a Random Forest model to predict land cover. .     
    1. #### Normalized Difference Vegetation Index (NDVI):
     NDVI is a metric used to assess vegetation health by measuring the difference between near-infrared (NIR) and red light reflected by plants. Healthy vegetation strongly reflects NIR and absorbs red light.
    **Formula**:  
    $$ \text{NDVI} = \frac{NIR - Red}{NIR + Red} $$  
    - Values range from **-1 to 1**:  
        - Near **1**: Dense, healthy vegetation.  
        - Near **0**: Sparse or non-vegetated areas (e.g., soil, rock).  
        - Negative values: Water, snow, or clouds.  
    Commonly used in agriculture, forestry, and environmental monitoring.
    2. #### Normalized Difference Built-Up Index (NDBI): 
    NDBI is a metric used to identify urban and built-up areas by leveraging the reflectance differences between short-wave infrared (SWIR) and near-infrared (NIR) bands. Built-up surfaces strongly reflect SWIR and absorb NIR.
    **Formula**:  
    $$ \text{NDBI} = \frac{SWIR - NIR}{SWIR + NIR} $$  
    - Values range from **-1 to 1**:  
        - Near **1**: Dense built-up areas (e.g., cities, roads).  
        - Near **0**: Bare soil or mixed surfaces.  
        - Negative values: Vegetation or water bodies.  
    Commonly used in urban planning, land-use studies, and monitoring urbanization. Derived from satellite data (e.g., Landsat, Sentinel-2).   
    3. #### Normalized Difference Water Index (NDWI)
    NDWI is a metric used to detect water bodies and monitor vegetation water content by comparing reflectance in the green and near-infrared (NIR) bands. Water strongly reflects green light and absorbs NIR.
    **Formula**:  
    $$ \text{NDWI} = \frac{Green - NIR}{Green + NIR} $$  
    - Values range from **-1 to 1**:  
        - Near **1**: Open water bodies (e.g., lakes, rivers).  
        - Near **0**: Dry soil or sparse vegetation.  
        - Negative values: Non-water features (e.g., built-up areas, dense vegetation).  
    Commonly used in hydrology, flood mapping, drought monitoring, and wetland studies. Derived from satellite data (e.g., Landsat, Sentinel-2).  
    **Note**: Variations like **MNDWI** (Modified NDWI) may use short-wave infrared (SWIR) instead of NIR for improved water detection.
    4. #### Built-Up Index (BUI) for Urban Density Detection
    BUI is a spectral index optimized to enhance identification of built-up areas (e.g., urban zones, impervious surfaces) by leveraging reflectance differences in SWIR, visible red, NIR, and blue bands. Built-up features exhibit higher reflectance in SWIR (B11) and red (B4), while vegetation/water absorb NIR (B8) and blue (B2).
    **Formula**:  
    $$ \text{BUI} = \frac{B11 + B4 - B8 - B2}{B11 + B4 + B8 + B2} $$  
    - **Value Interpretation**:  
        - **Near 1**: High-density urban areas (e.g., concrete, buildings).  
        - **Near 0**: Bare soil or mixed land cover.  
        - **Negative values**: Vegetation, water, or natural landscapes.  
    5. #### Urban Index  
    A simplified metric to identify urban/built-up areas by combining reflectance from SWIR, red, NIR, and blue bands. Urban surfaces typically reflect strongly in SWIR (B11) and red (B4), while absorbing NIR (B8) and blue (B2).  
    **Formula**:  
    $$ \text{Urban Index} = \frac{B11 + B4 - B8 - B2}{B11 + B4 + B8 + B2} $$  
    - **Values range from -1 to 1**:  
        - Near **1**: Dense urban areas (e.g., buildings, roads).  
        - Near **0**: Bare soil or mixed land cover.  
        - Negative values: Vegetation, water, or natural landscapes.  
    Used for mapping urbanization, infrastructure growth, and land-use changes. Compatible with Landsat or Sentinel-2 data.  
    6. #### Mean of Selected Bands (B2,B3,B4,B8,B11):
    In addition to the indices computed as mentioned above, the timeseriese mean of selected bands were also processed and added to the composite image to give a better picture of agriculture, builtup land cover.

    7.  #### Agriculture Pixel Percentage:
    A pixel mask of 0.2 was used to count the individual pixels having NDVI greater than 0.2. This count will be used to calcuated the percentage of agriculture pixels.
    
    8. #### Builtup Pixel Percentage:
    Similarly a pixel mask of 0.0 was used to count the individual pixels having value greater than 0.0. This count will be used to calcualte the percentage of builtup pixels. 

    9. #### Composite Image:
    Using Google Earth Engine (GEE) a composite image from the mean of means of features of 419 images. Thus the single composite image is the representation of 419 image collections with the mean of all the calculated featrures. Utilizing the power of GEE the feature set of this single composite image is reduced to the individual parcels pundaries. In this way we not have the featureset against every parcel geometry. This feature set is exported to geodataframe for futher precessing.   

### Landuse Hybrid Classification:
Utilizing the feature set calculated against each parcel, a hybrid classification approcach is adopted to divide the initial 5759 agriculture parcels into more meaning classes, which are relavenet with current ground condition. Therefore, these four land uses classes were used to classify the parcels
- Pure Agriculture: **2544**
- Partial Agriculture (25-75%) : **1790**
- Pure Builtup: **778**
- Partial Builtup (25-75%): **647**
These four classe were visualized on map and found correct to a greater extent with some exceptions. 

### Landuse Classification using Machine Learning:
In the Landuse Hybrid Classification approach we have used human element to set conditions based on the feature set for the classification of land use. It was necessary at that stage because we were not having a reliable label dataset to directly feed to the machine. So utilizing the hybrid classification approach we generated a reliable(to a greater extent) labeled dataset of the four land use classes to process and make it ready for machine learning. During processing we have to make remove the unnecessary colums, that we utilized in our hybrid classification approach. We have to remove the geometry column also from our dataset to train the model on pure features dataset without any reference to geometry of the these four land classes. The clean dataset is not divided into X,y for features and labels as

```
X = gdf[['B2','B3','B4','B8','B11', 'NDVI','NDBI', 'BUI', 'UI', 'NDWI']]

y = gdf['landuse_class_encoded']

```

Now the dataset is ready we divided into 30% -70%  for training and testing of the model.

```
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

```
#### Training and Testing the Model:
An instance (rf_model) of the RandomForestClassifire model is created with the following parapmeters
```
rf_model = RandomForestClassifier(n_estimators=100, class_weight='balanced' ,max_depth=15, min_samples_split=2,random_state=42)
rf_model.fit(X_train, y_train)
```
rf_model is then trained on the training dataset (30%) of the whole data. Once the model is trained then it is the time for prediction. Now the model is provided with the X_test features to make prediction.
```
y_pred = rf_model.predict(X_test)

```
This y_pred will be used to generate pixels based classification of land use. 

### Performance Evaluation :
Classification report and Confusion Matrix were calculated to print and analyze the performance. The classification report shows an 84% f1-score.
```
Classification Report:
                      precision    recall  f1-score   support

Partial-Agriculture       0.75      0.80      0.77       537
    Partial-Builtup       0.85      0.71      0.77       194
   Pure-Agriculture       0.88      0.87      0.88       763
       Pure-Builtup       0.95      0.98      0.96       234

           accuracy                           0.84      1728
          macro avg       0.86      0.84      0.85      1728
       weighted avg       0.85      0.84      0.84      1728

Confusion Matrix:
 [[428  19  90   0]
 [ 45 137   0  12]
 [ 98   0 665   0]
 [  0   5   0 229]]

```
### End Product: 
The final prediction is feed to the rasterio to write a landuse classification image. The image can be visualized with leafmap for interactive analysis and inspection.

 ![Fig](./sample_image-2.png)
 
 ### Uses and Wayforward
- Crop classification
- Urban Planning
- Land Use Policy. 

### Revised Version.
# Land Cover Classification: A Case Study of Five Mauzas in Tehsil Pabbi, Nowshera, Pakistan  

## Study Area  
This study focuses on five mauzas (revenue estates) in Tehsil Pabbi, District Nowshera, Khyber Pakhtunkhwa, Pakistan:  
1. Khushmaqam  
2. Khudrezi  
3. Chowki Mamraiz  
4. Chand Bibi  
5. Amankot  

## Dataset Overview  
The original land use designations in the `Landuse_Ma` column of the massavi (revenue record) are outdated and inconsistent with current ground conditions. Key statistics from the dataset are summarized below:  

| Landuse_Ma     | Parcel Count | Total Area (Acres) |  
|----------------|--------------|--------------------|  
| Agriculture    | 5,759        | 3,878.35           |  
| Built-up       | 24           | 73.87              |  
| Graveyard      | 29           | 25.20              |  
| Other          | 229          | 130.85             |  
| Road/Streets   | 172          | 207.78             |  
| Stream         | 302          | 102.74             |  
| **Total**      | **6,515**    | **4,419**          |  

### Key Data Anomalies  
1. **Misclassification of Agriculture Parcels**:  
   - Parcels labeled "Agriculture" now include built-up areas.  
   - Mixed-use parcels (agriculture + built-up) require reclassification.  
2. **Underrepresentation of Agriculture**:  
   - Active agricultural parcels are erroneously labeled as "Other."  

## Objectives  
To develop a clean agricultural parcel dataset for training crop classification models (e.g., Random Forest, SVM) by:  
1. Filtering misclassified parcels.  
2. Reclassifying land use using satellite-derived indices.  

---

## Methodology  

### Data Pipeline  
![Figure 1: Land Cover Classification Pipeline](./Model-V2.png)  

### Phase 1: Satellite Imagery Acquisition & Processing  
**Dataset**: Harmonized Sentinel-2 Surface Reflectance ([COPERNICUS/S2_SR_HARMONIZED](https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2_SR_HARMONIZED))  
**Temporal Range**: 2022–2024 (419 images, <10% cloud cover)  
**Processing Workflow**:  

1. **Feature Extraction**:  
   Computed spectral indices for all images and aggregated temporal means.  
   - **Composite Image**: Generated from the mean of 419 images.  
   - **Band Selection**: B2 (Blue), B3 (Green), B4 (Red), B8 (NIR), B11 (SWIR).  

2. **Spectral Indices**:  

#### 2.1 Normalized Difference Vegetation Index (NDVI)  
$$ \text{NDVI} = \frac{\text{NIR} - \text{Red}}{\text{NIR} + \text{Red}} $$  
- **Interpretation**:  
  - **>0.6**: Dense vegetation  
  - **0.2–0.6**: Sparse vegetation/agriculture  
  - **<0.2**: Non-vegetated  

#### 2.2 Normalized Difference Built-Up Index (NDBI)  
$$ \text{NDBI} = \frac{\text{SWIR} - \text{NIR}}{\text{SWIR} + \text{NIR}} $$  
- **Interpretation**:  
  - **>0**: Built-up areas  
  - **<0**: Vegetation/water  

#### 2.3 Normalized Difference Water Index (NDWI)  
$$ \text{NDWI} = \frac{\text{Green} - \text{NIR}}{\text{Green} + \text{NIR}} $$  
- **Interpretation**:  
  - **>0.2**: Water bodies  
  - **<0**: Non-aqueous surfaces  

#### 2.4 Built-Up Index (BUI)  
$$ \text{BUI} = \frac{\text{B11} + \text{B4} - \text{B8} - \text{B2}}{\text{B11} + \text{B4} + \text{B8} + \text{B2}} $$  
- **Threshold**: Values >0.3 indicate urban density.  

#### 2.5 Pixel Classification  
- **Agriculture Mask**: NDVI >0.2  
- **Built-up Mask**: NDBI >0.0  

---

### Phase 2: Hybrid Land Use Classification  
Initial filtering of 5,759 agricultural parcels yielded four classes:  

| Class                 | Parcel Count | Criteria                   |  
|-----------------------|--------------|----------------------------|  
| Pure Agriculture      | 2,544        | >75% agricultural pixels   |  
| Partial Agriculture   | 1,790        | 25–75% agricultural pixels |  
| Pure Built-up         | 778          | >75% built-up pixels       |  
| Partial Built-up      | 647          | 25–75% built-up pixels     |  

---

### Phase 3: Machine Learning Classification  

#### Training Configuration  
```
# Feature/Label Definition  
X = gdf[['B2','B3','B4','B8','B11', 'NDVI','NDBI', 'BUI', 'UI', 'NDWI']]  
y = gdf['landuse_class_encoded']  

# Train-Test Split (70:30)  
X_train, X_test, y_train, y_test = train_test_split(  
    X, y, test_size=0.3, random_state=42, stratify=y  
)  

# Model Initialization  
rf_model = RandomForestClassifier(  
    n_estimators=100,  
    class_weight='balanced',  
    max_depth=15,  
    min_samples_split=2,  
    random_state=42  
)  
rf_model.fit(X_train, y_train)  
```
#### Performance Evaluation
 ```
Classification Report:  
                      precision    recall  f1-score   support  

Partial-Agriculture       0.75      0.80      0.77       537  
    Partial-Builtup       0.85      0.71      0.77       194  
   Pure-Agriculture       0.88      0.87      0.88       763  
       Pure-Builtup       0.95      0.98      0.96       234  

           accuracy                           0.84      1728  
          macro avg       0.86      0.84      0.85      1728  
       weighted avg       0.85      0.84      0.84      1728  

Confusion Matrix:  
[[428  19  90   0]  
 [ 45 137   0  12]  
 [ 98   0 665   0]  
 [  0   5   0 229]]  


 ```

 ### End Product: 
The final prediction is feed to the rasterio to write a landuse classification image. The image can be visualized with leafmap for interactive analysis and inspection.

 ![Fig](./sample_image-2.png)

### Uses and Wayforward
- Crop classification
- Urban Planning
- Land Use Policy. 

In [115]:
#Step 1: Load the original dataset having Landuse_Ma columon. This dataset also have crop information. 

import geopandas as gpd
import pandas as pd
import numpy as np
import leafmap
import geemap
import ee

In [116]:
cropfile = 'D:\Trainings\gee\pabb_crop_V3.geojson'
gdf = gpd.read_file(cropfile)
gdf['Landuse_Ma'].groupby(gdf['Landuse_Ma']).count().sort_values(ascending=False)

Landuse_Ma
Agriculture     5759
Stream           302
Other            229
Road/Streets     172
Graveyard         29
Built up          24
Name: Landuse_Ma, dtype: int64

In [76]:
#len(gdf)
#gdf.groupby(['Landuse_Ma','Area_Acre']).sum().reset_index(name='count')
#gdf
#To sum the Area_Acre grouped by Landuse_Ma
# Sum the Area_Acre column grouped by Landuse_Ma 

#df = gdf.groupby('Landuse_Ma')['Area_Acre'].sum().reset_index(name='total_area_acre')
#df['parcel_count'] = gdf.groupby('Landuse_Ma')['Landuse_Ma'].count().values

#df['total_area_acre'] = df['total_area_acre'].round(2)

#display columns of df in parcel_count, total_area_acre
#df[['Landuse_Ma', 'parcel_count', 'total_area_acre']]

#df['total_area_acre'].sum()


# Add a count of parcels per Landuse_Ma
# To show Mauza wise Landuse_Ma
#df = gdf.groupby(['Landuse_Ma','Mouza_Name']).size().reset_index(name='count')
#df

np.float64(4418.789999999999)

In [117]:

selected_parcels = gdf[gdf['Landuse_Ma'] == 'Agriculture']
print(f'Total Agriculture Parcels are :{len(selected_parcels)}') 
# These are the parcels with Landuse_Ma as Agriculture and will be used for further analysis.

Total Agriculture Parcels are :5759


## Data Cleansing
1. Calculating NDVI and NDBI of the selected agriculture parcels : 5759.

In [118]:
# Lets Convert the gdf of the original file to ee features utilizing earth engine power
ee.Authenticate()

ee.Initialize()
boundary = geemap.gdf_to_ee(gdf) # this is for cliping the images
selectedparcels_fc = geemap.gdf_to_ee(selected_parcels) # this is for getting the agriculture parcels features Collection 


In [119]:
# Define a function to get imagery an apply filter
def get_collection (start_date,end_date):
    collection = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
    filtered_collection = collection.filterDate(start_date, end_date).filterBounds(boundary.geometry()).filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10))
    return filtered_collection

In [120]:
# defing ndvi , NDBI , BUI and UI indices

def get_ndvi(filtered_collection):
    # Calculate the annual NDVI from the filtered collection (filtered_collection)
    ndvi_collection = filtered_collection.map(lambda img: img.normalizedDifference(['B8', 'B4']).rename('NDVI'))
    # Calculate the mean NDVI for the year
    mean_ndvi = ndvi_collection.mean()
    return mean_ndvi
    
def get_ndbi(filtered_collection):
    # Calculate Normalized Difference Built-up Index (NDBI) (SWIR−NIR)/(SWIR+NIR) from the filtered collection filtered_collection
    # NDBI = (B11 - B8) / (B11 + B8)
    # Built‑up areas (positive values) vs. vegetation/water (negative)
    ndbi_collection = filtered_collection.map(lambda img: img.normalizedDifference(['B11', 'B8']).rename('NDBI'))
    # Calculate the mean NDBI for the year
    mean_ndbi = ndbi_collection.mean()
    return mean_ndbi
def get_ndwi(filtered_collection):
    # Calculate Normalized Difference Water Index (NDWI) (Green-NIR)/(Green+NIR) from the filtered collection filtered_collection
    # NDWI = (B3 - B8) / (B3 + B8)
    # Water (positive values) vs. built-up/vegetation (negative)
    ndwi_collection = filtered_collection.map(lambda img: img.normalizedDifference(['B3', 'B8']).rename('NDWI'))
    # Calculate the mean NDWI for the year
    mean_ndwi = ndwi_collection.mean()
    return mean_ndwi

def get_bui(filtered_collection):
    # Calculate the annual Built-Up Index (BUI) for an ee.ImageCollection `filtered_collection`.
    # BUI: (B11 + B4 - B8 - B2) / (B11 + B4 + B8 + B2)
    # Enhances urban density separation from other land‑covers
    bui_collection = filtered_collection.map(lambda img: (img.select('B11').add(img.select('B4')).subtract(img.select('B8'))
           .subtract(img.select('B2'))
           .divide(
               img.select('B11').add(img.select('B4'))
                  .add(img.select('B8'))
                  .add(img.select('B2'))
           )
           .rename('BUI')      
                    ))
    mean_bui = bui_collection.mean()
    return mean_bui
    
def get_ui(filtered_collection):
    # Calculate Urban Index (UI) (NDBI−NDVI)/(NDBI+NDVI) for the collection
    # Separates built‑up from vegetation more robustly
    mean_ndvi = get_ndvi(filtered_collection)
    mean_ndbi = get_ndbi(filtered_collection)
    mean_ui = mean_ndbi.subtract(mean_ndvi).divide(mean_ndbi.add(mean_ndvi)).rename('UI')
    return mean_ui



In [121]:
def get_composite(filtered_collection,ndvi_img, ndbi_img, bui_img, ui_img,ndwi_img):
    """
    Stack the four annual‐mean index images into one composite,
    then clip it to the boundary.
    """
    selected_bands = ['B2', 'B3', 'B4', 'B8', 'B11']
    filtered_collection = filtered_collection.select(selected_bands)
    
    #Now add the indices to each image in the filtered collection
    filtered_collection = filtered_collection.map(lambda img: img.addBands(ndvi_img).addBands(ndbi_img).addBands(bui_img).addBands(ui_img).addBands(ndwi_img))  
    
    # Reduce the collection to a mean single image
    composite_img = filtered_collection.mean()
    
    # Rename the bands in the correct order
    composite_img = composite_img.rename(['B2', 'B3', 'B4', 'B8', 'B11', 'NDVI', 'NDBI', 'BUI', 'UI','NDWI'])
    
    # 3. Clip to your boundary
    composite_img = composite_img.clip(boundary.geometry())
    
    return composite_img


In [122]:
filtered_collection = get_collection('2022-01-01', '2024-12-31')
ndvi_img = get_ndvi(filtered_collection)
ndbi_img = get_ndbi(filtered_collection)
bui_img = get_bui(filtered_collection)
ui_img = get_ui(filtered_collection)
ndwi_img = get_ndwi(filtered_collection)

composite_img = get_composite(filtered_collection, ndvi_img, ndbi_img, bui_img, ui_img, ndwi_img)

In [95]:
Map = geemap.Map(center=(37.5,70), zoom=10)

# To confirm that the composite Image is cropped 
vis_tc = {
    'bands': ['B4','B3','B2'],  # Red, Green, Blue
    'min': 0, 'max': 3000,      # Sentinel-2 SR values ~0–3000
    'gamma': 1.2
}
Map.addLayer(composite_img, vis_tc, 'True Color Composite', True)

Map.center_object(boundary, 10)
Map


Map(center=[34.025104580270586, 71.77439126406186], controls=(WidgetControl(options=['position', 'transparent_…

## Create a binary mask for each pixel in the composite.
1. if the NDVI value of a pixel is greater than 0.25 it is agricultue pixel
2. if the NDBI value of a pixel is greater than 0.0 than it is a builtup pixel


In [85]:
#builtmask = composite_img.select('BUI').gt(0)
#agrimask = composite_img.select('NDVI').gt(0.25)


In [123]:
def sum_count(filtered_collection, parcels_fc,agrithresh=0.3, builtdiff =0.0):
        
    flagged = filtered_collection.map(lambda img: mask_flags(img, agrithresh, builtdiff))
    
    count_image = flagged.select(['agri_count','built_count']).sum()

    stats_fc = count_image.reduceRegions(
        collection=parcels_fc,
        reducer=ee.Reducer.sum(),
        scale=10,
        crs=count_image.projection()
    )
    return stats_fc


def mask_flags(img,agrithresh, builtdiff):
    
    #compute the NDVI and NDBI for each image in the collection
    ndvi = img.normalizedDifference(['B8', 'B4'])
    ndbi = img.normalizedDifference(['B11', 'B8'])
    # create a boolean mask and cast to int (1 if true and 0 if false)
    agri_flag = ndvi.gt(agrithresh).rename('agri_count').toInt()
    built_flag = ndbi.gt(builtdiff).rename('built_count').toInt()
    return img.addBands([agri_flag,built_flag])


In [124]:
# The composite Image which is mean of all the indices and bands is now used to 
# calculate the mean of each index and bands in the composite image.

def mean_indices(composite_img, parcels_fc):
    #Select the bands of interest
    b2 = composite_img.select('B2')
    b3 = composite_img.select('B3')
    b4 = composite_img.select('B4')
    b8 = composite_img.select('B8')
    b11 = composite_img.select('B11')


    # Select the indices
    ndvi = composite_img.select('NDVI')
    ndbi = composite_img.select('NDBI')
    ui = composite_img.select('UI')
    bui = composite_img.select('BUI')
    ndwi = composite_img.select('NDWI')

    index_img = ee.Image.cat([b2,b3,b4,b8,b11,ndvi,ndbi,bui,ui,ndwi])
    mean_img = index_img.reduceRegions(
        collection=parcels_fc,
        reducer=ee.Reducer.mean(),
        scale=10,
        crs=composite_img.projection()
    )
    return mean_img


In [125]:
pxlsum = sum_count(filtered_collection,selectedparcels_fc,agrithresh=0.3, builtdiff =0.0)
pxlmean = mean_indices(composite_img, selectedparcels_fc)
# Convert the ee.FeatureCollection to a pandas DataFrame
pxlsum_gdf= geemap.ee_to_gdf(pxlsum)
pxlmean_gdf= geemap.ee_to_gdf(pxlmean)  


In [126]:
pxlmean_gdf.head()

,geometry,Area_Acre,B11,B2,B3,B4,B8,BUI,Crop_Type,FFID,Landuse_Ma,Mouza_Name,NDBI,NDVI,NDWI,Parcel_ID,UI
0,"POLYGON ((71.74996 34.03193, 71.75027 34.03195...",0.101785,2147.930654,1080.794583,1314.424346,1412.001354,2250.015724,0.037398,None,1,Agriculture,Khushmaqam,-0.016614,0.226656,-0.261929,668,-1.202087
1,"POLYGON ((71.74868 34.03205, 71.74869 34.03195...",0.315557,2148.237869,905.980655,1133.265953,1248.493593,2115.168267,0.060771,None,3,Agriculture,Khushmaqam,0.013744,0.254974,-0.298905,632,-0.876391
2,"POLYGON ((71.75021 34.03218, 71.75022 34.0321,...",0.187644,2198.647527,989.403836,1234.108843,1331.561342,2231.942662,0.047527,None,4,Agriculture,Khushmaqam,-0.003842,0.248897,-0.284861,669,-1.022695
3,"POLYGON ((71.75401 34.03224, 71.75403 34.03219...",0.161568,2193.297278,806.999988,1099.118312,1177.751882,2761.666157,-0.038227,None,5,Agriculture,Khushmaqam,-0.109844,0.410130,-0.431634,693,-1.738528
4,"POLYGON ((71.74971 34.03192, 71.74996 34.03193...",0.229336,2126.134001,1160.127481,1376.297936,1480.144814,2196.267099,0.039058,None,6,Agriculture,Khushmaqam,-0.011919,0.197473,-0.233127,667,-1.196795


In [127]:
# since both dataframes have duplicate columns therefore to avoid duplicaiton 
# we will select the columns we need for merging
mean_cols = ['geometry','NDVI','NDBI','BUI','UI','NDWI','B2','B3','B4','B8','B11']
plxmean_small = pxlmean_gdf[mean_cols]

#Now we will merge the two dataframes
stats_gdf = pxlsum_gdf.merge(plxmean_small, on='geometry', how='left')

print(stats_gdf.columns.tolist())

['geometry', 'Area_Acre', 'Crop_Type', 'FFID', 'Landuse_Ma', 'Mouza_Name', 'Parcel_ID', 'agri_count', 'built_count', 'NDVI', 'NDBI', 'BUI', 'UI', 'NDWI', 'B2', 'B3', 'B4', 'B8', 'B11']


In [128]:
# Now stats_gdf has columns 'built_count' and 'agri_count' per parcel
# Lets compute the percentage of built and agri land
stats_gdf['pct_built'] = stats_gdf['built_count'] / (stats_gdf['built_count'] + stats_gdf['agri_count'])
stats_gdf['pct_agri']  = stats_gdf['agri_count']  / (stats_gdf['built_count'] + stats_gdf['agri_count'])


In [129]:
stats_gdf

,geometry,Area_Acre,Crop_Type,FFID,Landuse_Ma,Mouza_Name,Parcel_ID,agri_count,built_count,NDVI,...,BUI,UI,NDWI,B2,B3,B4,B8,B11,pct_built,pct_agri
0,"POLYGON ((71.74996 34.03193, 71.75027 34.03195...",0.101785,None,1,Agriculture,Khushmaqam,668,90.725490,181.058824,0.226656,...,0.037398,-1.202087,-0.261929,1080.794583,1314.424346,1412.001354,2250.015724,2147.930654,0.666186,0.333814
1,"POLYGON ((71.74868 34.03205, 71.74869 34.03195...",0.315557,None,3,Agriculture,Khushmaqam,632,445.384314,928.509804,0.254974,...,0.060771,-0.876391,-0.298905,905.980655,1133.265953,1248.493593,2115.168267,2148.237869,0.675823,0.324177
2,"POLYGON ((71.75021 34.03218, 71.75022 34.0321,...",0.187644,None,4,Agriculture,Khushmaqam,669,241.717647,430.784314,0.248897,...,0.047527,-1.022695,-0.284861,989.403836,1234.108843,1331.561342,2231.942662,2198.647527,0.640570,0.359430
3,"POLYGON ((71.75401 34.03224, 71.75403 34.03219...",0.161568,None,5,Agriculture,Khushmaqam,693,955.690196,434.231373,0.410130,...,-0.038227,-1.738528,-0.431634,806.999988,1099.118312,1177.751882,2761.666157,2193.297278,0.312414,0.687586
4,"POLYGON ((71.74971 34.03192, 71.74996 34.03193...",0.229336,None,6,Agriculture,Khushmaqam,667,170.831373,437.709804,0.197473,...,0.039058,-1.196795,-0.233127,1160.127481,1376.297936,1480.144814,2196.267099,2126.134001,0.719277,0.280723
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5754,"POLYGON ((71.76623 34.0439, 71.76626 34.04366,...",0.171500,None,6723,Agriculture,Amankot,843,1553.215686,112.615686,0.571489,...,-0.101405,-1.856395,-0.555056,460.745576,706.997455,669.423946,2584.963943,1786.159601,0.067603,0.932397
5755,"POLYGON ((71.76681 34.0447, 71.76711 34.04347,...",0.466952,None,6724,Agriculture,Amankot,848,2741.811765,1695.423529,0.420987,...,-0.035676,-1.744243,-0.461200,606.152732,866.504644,990.755175,2359.760189,1915.876354,0.382090,0.617910
5756,"POLYGON ((71.77762 34.02896, 71.77766 34.02877...",-0.303899,None,6725,Agriculture,Amankot,1331,3572.301961,813.478431,0.488737,...,-0.062202,-1.782541,-0.494530,601.428608,912.079541,938.331616,2769.259731,2078.009210,0.185481,0.814519
5757,"POLYGON ((71.79385 34.02814, 71.79411 34.02642...",3.634200,None,6726,Agriculture,Amankot,2070,45075.819608,22076.682353,0.417301,...,-0.031983,-1.711403,-0.453149,676.936347,959.914195,1077.387722,2598.212482,2088.983032,0.328754,0.671246


In [130]:
def classify_hybrid(row,
                    ndvi_ag=0.3, ui_built=0.2,
                    maj_pct=0.75, min_pct=0.50):
    ndvi = row['NDVI']
    ui   = row['UI']
    pa   = row['pct_agri']
    pb   = row['pct_built']
    
    # 1) Spectral “easy wins”
    
    if ndvi >= ndvi_ag and ui <= 0:
        return 'Pure-Agriculture'
    if ui >= ui_built:
        return 'Pure-Builtup'

    # 2) Percent-based majority
    if pa >= maj_pct:
        return 'Pure-Agriculture'
    if pb >= maj_pct:
        return 'Pure-Builtup'

    # 3) Partial by percent
    if pa >= min_pct and pa > pb:
        return 'Partial-Agriculture'
    if pb >= min_pct and pb > pa:
        return 'Partial-Builtup'
    if pa >= pb and pa < min_pct:
        return 'Barren'

    # 4) Everything else → Mixed
    return 'Mixed'

# Apply and inspect
stats_gdf['landuse_class'] = stats_gdf.apply(classify_hybrid, axis=1)
print(stats_gdf['landuse_class'].value_counts())


landuse_class
Pure-Agriculture       4482
Pure-Builtup            778
Partial-Builtup         426
Partial-Agriculture      73
Name: count, dtype: int64


In [131]:
pure_agri = stats_gdf[stats_gdf['landuse_class'] == 'Pure-Agriculture']
pure_built = stats_gdf[stats_gdf['landuse_class'] == 'Pure-Builtup']
partial_agri = stats_gdf[stats_gdf['landuse_class'] == 'Partial-Agriculture']
partial_built = stats_gdf[stats_gdf['landuse_class'] == 'Partial-Builtup']
#mixed = stats_gdf[stats_gdf['landuse_class'] == 'Mixed']


In [133]:
m = leafmap.Map(center=(37.5, 70), )
m.add_basemap('Google Satellite')

# 1. Create a minimal GeoDataFrame with only geometry + landuse_class
popup_gdf = stats_gdf[['geometry', 'landuse_class']].copy()


legend_dict = {
    'Pure Agriculture':           '#00AA00',  # dark green
    'Pure Builtup':               '#00008B',  # darkblue
    'Partial Agri(50-75%)':       '#90EE90',  # Light green
    'Partial Builtup(50-75%)':    '#ADD8E6',  # light blue
    #'Mixed':                      '#FF0000',  # red
}


m.add_gdf(
    popup_gdf,
    layer_name='Landuse Class',
    style={                      # pick any style you like
        'color':   'black',
        'fillColor':'transparent',
        'fillOpacity': 0.1,
        'weight':  1
    },
    zoom_to_layer=True,
    info_mode='on_click'        # show popup when you click
)

# Pure Agriculture
m.add_gdf(
    pure_agri,
    layer_name='Pure Agriculture',
    style={
        'color': 'green',      # border
        'fillColor': 'green',  # interior
        'fillOpacity': 0.4,
        'weight': 1
    }, info_mode='off'
     
)
# Pure Builtup
m.add_gdf(
    pure_built,
    layer_name='Pure Builtup',
    style={
        'color': 'blue',
        'fillColor': 'blue',
        'fillOpacity': 0.4,
        'weight': 1
    }, info_mode='off'
    
)

# Partial Agriculture
m.add_gdf(
    partial_agri,
    layer_name='Partial Agri (50-75%)',
    style={
        'color': 'lightgreen',
        'fillColor': 'lightgreen',
        'fillOpacity': 0.4,
        'weight': 1
    }, info_mode='off'
    
)
# Partial Builtup
m.add_gdf(
    partial_built,
    layer_name='Partial Builtup(50-75%)',
    style={
        'color': 'lightblue',
        'fillColor': 'lightblue',
        'fillOpacity': 0.4,
        'weight': 1
    }, info_mode='off'
    
)

# Add legend
m.add_legend(
    legend_title='Landuse Classification',
    legend_dict=legend_dict,
    position='bottomleft'
)

m.add_layer_manager()

m

Map(center=[37.5, 70], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_t…

In [134]:
len(stats_gdf)

5759

In [135]:
stats_gdf.to_file('Pabbi_RevisedLandtype-V4.geojson')

In [ ]:
m = leafmap.Map(center=(37.5, 70), zoom=6)
m.add_basemap('Google Satellite')

# 1. Create a minimal GeoDataFrame with only geometry + landuse_class
popup_gdf = stats_gdf[['geometry', 'landuse_class']].copy()

# 2. Add your styling layers *without* popups
layers_lsit = [ 
    (pure_agri,    'Pure Agriculture',    legend_dict['Pure Agriculture']),
    (pure_built,   'Pure Builtup',        legend_dict['Pure Builtup']),
    (partial_agri, 'Partial Agriculture', legend_dict['Partial Agri(50-75%)']),
    (partial_built,'Partial Builtup',     legend_dict['Partial Builtup(50-75%)']),
   # (mixed,        'Mixed',               legend_dict['Mixed']),
        ]

for gdf, name, color in layers_lsit:
    # 2.1 Add the layer to the map   
    m.add_gdf(
        gdf,
        layer_name=name,
        style={
            'color':    color,
            'fillColor':color,
            'fillOpacity':0.3,
            'weight':   1
        },
        zoom_to_layer=True,
        info_mode='off'   # make sure no popups for these layers
    )

# 3. Add your minimal layer *last*, with popups ON only for landuse_class
m.add_gdf(
    popup_gdf,
    layer_name='Landuse Class',
    style={
        'color':      'black',
        'fillColor': 'transparent',
        'fillOpacity':0.1,
        'weight':     1
    },
    info_mode='on_click'  # only this layer shows a popup
)

# 4. Legend & Layer Manager
m.add_legend(
    legend_title='Landuse Classification',
    legend_dict=legend_dict,
    position='bottomleft'
)

# Adding the land classification raster
colorlist = ['lightgreen','lightblue','darkgreen','darkblue']
classified_img = 'D:/Trainings/gee/classified_image-V2-clipped.tif'



m.add_raster(
    classified_img,
    layer_name='Landuse Classification',
    colormap= colorlist,
    opacity=0.5,
    vmin=0,
    vmax=4,
    nodata=None,
    legend=True,
    layer_opacity=0.5
)

m.add_layer_manager()

m


Map(center=[34.016954, 71.7731905], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title'…